スピン配位の生成と保存 
lattice size: L = 16
inverse teemperature: β=0.04, 0.08, ・・・, 1.0

In [1]:
# ライブラリのインポート
import numpy as np
import random
import tqdm

nsp = 16   # model size 

In [2]:
import glob
import os
import re

# ファイルに自動的に連番を付ける関数
def make_filename_by_seq(dirname, filename, seq_digit=3):
    filename_without_ext, ext = os.path.splitext(filename)
    
    pattern = f"{filename_without_ext}_([0-9]*){ext}"
    prog = re.compile(pattern)

    files = glob.glob(
        os.path.join(dirname, f"{filename_without_ext}_[0-9]*{ext}")
    )

    max_seq = -1
    for f in files:
        m = prog.match(os.path.basename(f))
        if m:
            max_seq = max(max_seq, int(m.group(1)))
    
    new_filename = os.path.join(dirname, f"{filename_without_ext}_{max_seq+1:0{seq_digit}}{ext}")
    
    return new_filename

Metropolis MC 法

In [3]:
# 初期配位から n_th ステップまで配位をアップデートし最初の平衡配位とする
# その後 n_MC 回アップデートするごとに平衡配位をピックアップ
n_th = 1000000
#n_data = 10000    # train data 作成時
#n_data = 1000     # test data 作成時
n_data = 10       # 測定資料作成時
n_MC = 10000
n_loop = n_th+n_data * n_MC

# T = T_i から　T＝T_f まで dT=(T_f-T_i)/n_T 刻みでデータを生成
beta_i = 0.04
beta_f = 1.0
n_beta = 25
dbeta =beta_f/n_beta

# 磁化が負の場合にすべてのスピンを反転
def reverse(Spin):
    Mag = Spin.sum()
    if Mag <0:
        Spin = -Spin
    return Spin

for n_temp in range(n_beta):
    # temperature and its inverse
    random.seed(n_temp*n_data)   # 訓練データとテストデータで異なる乱数の設定
    beta = beta_i+n_temp * dbeta
    K =beta
    #　initialization by random spin configuration（温度を変えるごとにスピン配位をランダムに初期化)
    spin = np.array([[(2*random.randint(0,1)-1) for i in range(nsp)] for j in range(nsp)])

    for n in tqdm.tqdm(range(n_loop)):    # 進捗状況のバー表示
        i = random.randint(0,nsp-1)
        j = random.randint(0,nsp-1)
        En = K*(spin[(i-1)%nsp,j] + spin[(i+1)%nsp,j]+spin[i,(j+1)%nsp]+spin[i,(j-1)%nsp])*spin[i,j]
        p = np.exp(-2*En)
        r = np.random.uniform()
        if r < p:
            spin[i,j] = -spin[i,j]
        if (n == n_th) and (n_temp == 0):
            spin = reverse(spin)
            spin_data = np.append(spin, beta)    # 最初の熱平衡状態の配位を spin_data に登録
        elif (n >= n_th) and (n % n_MC == 0):
            spin = reverse(spin)
            s_temp = np.append(spin, beta)
            spin_data = np.append(spin_data, s_temp) # n_MC 毎にスピン配位をspin_data に追加

path_w = make_filename_by_seq("Data", "spin_data")
np.save(path_w, spin_data)

100%|█████████████████████████████████████████████████████████████████████| 1100000/1100000 [00:12<00:00, 87701.12it/s]


In [ ]:
spin_data.shape